# LAPACK via Julia

See [here](https://docs.julialang.org/en/stable/stdlib/linalg/#LAPACK-Functions-1) for documentation of Julia's LAPACK interface.  There is more than what we see here.

# Solving a Linear System

Here, we see how to solve `A x = b` for x.  This uses LAPCK's `gesv` function.

In [24]:
# setup square system
m = 50
n = 50
A = randn(m,n)
b = randn(m)
;

In [32]:
# solve using default backslash
@time x = A\b
@show norm(A*x - b)
;

  0.008339 seconds (12 allocations: 20.953 KiB)
norm(A * x - b) = 3.1145164996471335e-14


In [ ]:
A0 = copy(A)
b0 = copy(b)

In [33]:
A = copy(A0)
b = copy(b0)
# gesv - solve linear system with general matrix
# overwrites A with its LU factorization
# overwrites b with solution x
@time LAPACK.gesv!(A, b) 
@show norm(A0*b - b0)
@show norm(b - x)
;

  0.007570 seconds (7 allocations: 752 bytes)
norm(A0 * b - b0) = 4.0429393090254516e-14
norm(b - x) = 17.032199190832504


You can also solve multiple right hand sides by stacking them into a matrix

In [49]:
B0 = randn(m,3) # 3 RHS
;

In [51]:
B = copy(B0)
A = copy(A0)
@time LAPACK.gesv!(A,B)
@show norm(A0*B - B0)
;

  0.013350 seconds (7 allocations: 752 bytes)
norm(A0 * B - B0) = 8.33825480907231e-14


# Linear Least Squares

This uses `gels`.  We'll use standard statistical notation instead of Householder notation

In [78]:
n = 100
p = 50
X0 = randn(n,p)
β0 = randn(p)
y0 = X0*β0 # create exact solution
# min_x ∥Xβ - y∥^2
# gels - linear least squares
# Overwrites X with QR factorization 
# Overwrites y with solution β
X = copy(X0)
y = copy(y0)
@time LAPACK.gels!('N', X, y)
β = y[1:50]
@show norm(X0*β - y0)
@show norm(β - β0)
;

  0.000221 seconds (16 allocations: 53.328 KiB)
norm(X0 * β - y0) = 3.195128509104355e-14
norm(β - β0) = 3.745781777859878e-15


# Eigenvalues and Eigenvectors

In [38]:
n = 50
A = randn(n,n)
S = A*A'
S0 = copy(S)
;

In [39]:
?LAPACK.syev!

```
syev!(jobz, uplo, A)
```

Finds the eigenvalues (`jobz = N`) or eigenvalues and eigenvectors (`jobz = V`) of a symmetric matrix `A`. If `uplo = U`, the upper triangle of `A` is used. If `uplo = L`, the lower triangle of `A` is used.


In [79]:
S = copy(S0)
@time (E, V) = LAPACK.syev!('V', 'L', S)
# we see that S gets over-written with eigenvectors
@show norm(S - V)
;

  0.001362 seconds (11 allocations: 14.328 KiB)
norm(S - V) = 0.0


# Orthogonal matrices

Sometimes it is advantageous to have an orthogonal basis for the range of a matrix.  This comes up in subspace iteration, and randomized linear algebra

You can do this in-place using LAPACK functions.

In [30]:
# naive way to generate random orthogonal matrix
function orth_naive(m::Int, n::Int)
    A = randn(m,n)
    Q, R = qr(A)
    return Q
end

orth_naive (generic function with 1 method)

In [40]:
@time Q = orth_naive(m, n);

  0.000947 seconds (31 allocations: 185.797 KiB)


In [34]:
function orth_qr(m::Int, n::Int)
    A = randn(m,n)
    tau = Array{Float64}(min(m,n))
    LAPACK.geqrf!(A, tau)
    LAPACK.orgqr!(A, tau)
end

orth_qr (generic function with 1 method)

In [42]:
@time Q = orth_qr(m,n)
;

  0.000329 seconds (14 allocations: 65.328 KiB)
